In [231]:
import random
import math


class TableEntry:
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.in_table = True

    def get_key(self):
        return self.key

    def get_value(self):
        return self.value

    def set_value(self, value):
        self.value = value

    def is_in_table(self):
        return self.in_table

    def set_to_removed(self):
        self.key = None
        self.value = None
        self.in_table = False

    def is_removed(self):
        return (self.key is None) and (self.value is None) and not self.in_table



Hash Table With Count

In [253]:
class HashTableWithCount():
                                        #####
    def __init__(self, initial_capacity=(100)):     # We're gonna need min 100
                                        #####
        self.table_size = self.get_next_prime(initial_capacity) # make a table sized w a prime over 100
        self.table = [None] * self.table_size                   # fill it with None
        self.item_count = 0                 # a variable to track how many elements are added/removed
        self.total_probes = 0               # a variable to track probes for the experiment

    def get_next_prime(self, n):
        # uhh, let's use our existing prime methods
        candidate = (n+1)                       # starting with the next possible number
        while not self.is_prime(candidate):     # check if it's prime
            candidate += 1                      # if not, iterate
        return candidate                        # return the next prime found

    def isOdd(self, n):                         # a helper function to check if an integer is odd
        if((n%2)==0):
            return False
        else:
            return True
        
    # thank heck we made one of these earlier
    def is_prime(self, n):
        if(n==1 or n==0):                               # handles the cases of 1 and 0
            return False
        elif(n==2):                                     # handles the case of 2
            return True
        elif(not self.isOdd(n)):                        # handles all even numbers except 0 and 2
            return False
        for x in range(2,round(n**(1/2))+1):            # for every number x between 2 and sqroot n
            if(self.isOdd(x)):                          # only checks odds cause we already checked 2
                if((n%x)==0):                           # if n divides by x evenly
                    return False                        # return false
        return True                                     # if none divide evenly, it's a prime
    
    # def check_size(self, size): 
    # above is the original header; didn't end up using size though

    def check_size(self):                               # i'm not sure why size is passed to this
        size = self.table_size                          # size is table size
        ratio = (self.item_count/size)                  # we get a ratio of how full the array is
        return ratio                                    # return the ratio of count/size

    def get_hash_index(self, key):
        return (hash(key) % self.table_size)            # computes the first index of the given key

    def is_hash_table_too_full(self):
                              #####
        if(self.check_size() > 0.95):                    # checks if the hash table is 70% full
                              #####
            return True                                 # returns True, it is to full, if over 0.7
        else: return False                              # returns false if check returns 0.7 or less

    def enlarge_hash_table(self):
        # makes the table size the next prime number at least double the current size
                                                                  #####
        new_table_size = self.get_next_prime(round(self.table_size*1.05))
                                                                  #####
        old_table = self.table                              # grab the old table
        self.table = [None] * new_table_size                # make a new empty table
        self.table_size = new_table_size                    # set table_size variable
        self.item_count = 0                                 # reset itemcount before adding
        for entry in old_table:                             # for every entry in the old table
            if entry is not None and entry.is_in_table():   # if it's not None and not removed
                self.add(entry.get_key(), entry.get_value())# add it to the table

    def reset_probe_count(self):
        self.total_probes = 0                               # just sets probes to 0

    def get_probes(self):
        return self.total_probes                            # returns the probe total
    
    def get_item_count(self):
        return self.item_count                              # returns the item count

table = HashTableWithCount()

# testing enlarge function
#print(table.table_size)
#table.enlarge_hash_table()
#print(table.table_size)
#table.enlarge_hash_table()
#print(table.table_size)

#print(table.is_prime(20))
#print(table.isOdd(5))



Get Statistics

Linear Probing With Count

In [254]:
class LinearProbingWithCount(HashTableWithCount):

    # def probe(self, index, key)
    # above is the old header but I didn't need "key" in my implementation
    
    def probe(self, index):
        self.total_probes += 1                      # increment probe count
        return ((index + 1) % self.table_size)      # return the next index
    

    # def locate(self, index, key):
    # above is the old input; but I didn't need to pass index
    def locate(self, key):
        #self.total_probes += 1
        index = self.get_hash_index(key)            # grab the first possible index
        while self.table[index] is not None:        # stop if we hit an empty cell
            if self.table[index].get_key() == key:  # if the key is at this index
                return index                        # return the index it was found in
            index = self.probe(index)               # use probe to search further indices
        return None                                 # return none if we don't find the key
    

    def add(self,key,value):
        index = self.get_hash_index(key)            # grab the index
        while self.table[index] is not None:        # until we hit an empty
            if self.table[index].get_key() == key:  # if it's already in there
                self.table[index].set_value(value)  # update the value
                return                              # don't run forever
            index = self.probe(index)               # otherwise, probe til we find it
        self.table[index] = TableEntry(key,value)   # insert the new entry
        self.item_count += 1                        # increment num_items
        self.reset_probe_count()                    # make sure adding doesn't alter-
                                                    # test results by resetting
        if self.is_hash_table_too_full():           # check if the table is reaching capacity
            self.enlarge_hash_table()               # enlarge if needed


    def remove(self, key):
        index = self.locate(key)                    # pull the index of the given key using locate()
        if index is not None:                       # if the item is found
            self.table[index] = None                # remove the entry
            self.item_count -= 1                    # decrement item count
        else:                                       # if something goes wrong
            print("Linear remove operation failed: Key not found") # lmk


    def get_value(self, key):
        index = self.locate(key)                    # use locate to find the object
        if index is not None:                       # if entry is found
            entry = self.table[index]               # grab the found object
            return entry.get_value()                # use TableEntry's get_value method
        return None                                 # If the entry isn't found, return None
    

    def contains(self, key):
        index = self.locate(key)                    # locate the index
        if index is not None:                       # if we find the key
            return True                             # return True
        return False                                # else return False
    

    def is_empty(self):
        return (self.item_count==0)                 # True if 0 items, False otherwise
    

    def get_size(self):
        return self.table_size                      # returns the item count
    
    
    def clear(self):
        self.table = [None] * 101                   # empty the table, set it to default size
        self.table_size = 101
        self.item_count = 0                         # reset item count

Double Hashing With Count

In [255]:
class DoubleHashingWithCount(HashTableWithCount):

    
    def primary_hash(self, key):
        """First hash function."""
        return hash(key) % self.table_size
    

    def secondary_hash(self, key):
        """Second hash function to calculate step size."""
        return 1 + (hash(key) % (self.table_size - 1))
    

    def probe(self, index, key):
        # I've set probe up to do the stepping, while Locate and Add will interpret
        # the result returned by probe
        step = self.secondary_hash(key)             # finds the step length
        while (self.table[index] is not None) and (self.table[index].get_key() != key):
                                                    # until we find the key, or an empty bucket
            self.total_probes += 1                  # iterate probe count
            index = (index + step)%self.table_size  # we keep stepping
        return index                # we return the index of the given key or empty bucket
    

    # def locate(self, index, key): # don't think i'll need to pass index
    def locate(self, key):
        #self.total_probes += 1
        index = self.primary_hash(key)                  # grabs the first index
        while self.table[index] is not None:            # until we hit an empty
                if self.table[index].get_key() == key:  # if the key is in the bucket
                    return index                        # located, return the index
                index = self.probe(index,key)           # probe and grab the index
        return None                                     # if we hit an empty, return None
    

    def add(self,key,value):                            # same add method as Linear
        index = self.primary_hash(key)                  # grab the first index
        while self.table[index] is not None:            # until we hit an empty
            if self.table[index].get_key() == key:      # if it's already in there
                self.table[index].set_value(value)      # update the value
                return                                  # don't run forever
            index = self.probe(index,key)               # otherwise, probe til we find it
        self.table[index] = TableEntry(key,value)       # insert the new entry
        self.item_count += 1                            # increment num_items
        self.reset_probe_count()                        # make sure adding doesn't alter-
                                                        # test results by resetting
        if self.is_hash_table_too_full():               # check if the table is reaching capacity
            self.enlarge_hash_table()                   # enlarge if needed


    def remove(self, key):                          # same remove method should do the trick
        index = self.locate(key)                    # pull the index of the given key using locate()
        if index is not None:                       # if the item is found
            self.table[index] = None                # remove the entry
            self.item_count -= 1                    # decrement item count
        else:                                       # if something goes wrong
            print("Double hash remove operation failed: Key not found") # lmk


    def get_value(self, key):                       
        index = self.locate(key)                    # grab the appropriate index using locate
        if index is not None:                       # if we find the item
            return self.table[index].get_value()    # return its value
        return None                                 # return none if not found
    

    def contains(self, key):                        # same method as Linear should work
        index = self.locate(key)                    # locate the index
        if index is not None:                       # if we find the key
            return True                             # return True
        return False                                # else return False
    

    def is_empty(self):                             # checks if the table is empty
        return (self.item_count==0)                 # returns True if it's empty and False otherwise
    

    def get_size(self):
        return self.table_size                      # returns the item count
    

    def clear(self):                                # flips the table over and gets a new one
        self.table = [None] * 101                   # empty the table, set it to default size
        self.table_size = 101
        self.item_count = 0                         # reset item count



Name Generator:

In [256]:
# behold, a name generator way more complicated than it needed to be

def name_generator(n1,n2):   

    genList = []

    v = 1
    while (26**v)<(n1+n2):                  # find how many letters we need to satisfy both lists
        v += 1                              # increment v (the number of letters we'll use)
        
    while (len(genList) < (n1+n2)):         # till genList is as many elements as called for
        for j in range(2,v+1):              # generate all the names with j letters
            genList.extend(recursy(j))      # til we reach v letters
    
    listN1 = genList[:n1]                   # make the first n1 elements listN1
    listN2 = genList[n1:]                   # make the rest listN2
    listN2 = listN2[:n2]                    # chop the list to n2 length

    # Some diagnostic stuff:
    #print("listN1 length: ",len(listN1)) # for diag
    #print("listN2 length: ",len(listN2)) # for diag
    #print(listN1)
    #print(listN2)

    return (listN1,listN2)

def recursy(lCount = int, returnList = None, rCount = 0, name_string = ""):
    charlist = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r',
                's','t','u','v','w','x','y','z']
    # returns a list of all letter combinations with lCount number of characters
                                            # recurses to lCount depth
                                            # each appends their letter to name_string
    if returnList is None:                  # if there isn't yet a returnlist
        name_string = ""                    # set name string
        returnList = []                     # set return list

    for c in charlist:                      # for every char in the alphabet                 
        new_string = name_string + c        # append the char to the passed name_string
        rCount = rCount+1                   # iterate rCount
        if (rCount == lCount):              # if it's the deepest instance of recursion
            returnList.append(new_string)   # add completed name to return list
        else:                               # if it's not the deepest instance
                                            # call recursy again
            recursy(lCount, returnList, rCount, new_string)
        rCount -= 1                         # decrement rCount after recursion
                                            # once all the looping mumbo is done
    return returnList                       # return the list

    
# print(recursy(lCount=2))                  # testing
# print(name_generator(100,1000))           # testing

In [ ]:
import random   
import math
# we're allowed to import random and math right?
# both of these get imported in TableEntry and they're from the standard library, so I'm just going to assume

class GetStatistics:

    @staticmethod
    def main():                                             # this class does science

        # The results requested in the problem text in the form of an excerpt from my final notes at the end-
        # of the big long markdown that contains all the experiment data I thought was pertinent
        """
        The data I've collected suggests that the double hashing method is able to achieve nearly 1.5 PPS (Probes Per Search)
        with a table size of only 163, while the linear hash method needs a table of size 197 to get to 1.5 PPS.
        """
        # I talk in detail about why I think this might be in the notes at the bottom of the big markdown down there



        linear_ht = LinearProbingWithCount()                # Our linear hash table
        double_ht = DoubleHashingWithCount()                # our double hash table

        linear_count = []                                   # list for linear prob stats
        double_count = []                                   # list for double hash stats

        names = GetStatistics.generate_names()              # a list to hold 11k three letter names
        add_list = GetStatistics.get_1000_names(names)      # a list of names to add from
        search_list = GetStatistics.get_10000_names(names)  # a list of names to search for


        for t in range(1000):
            GetStatistics.choose_100_names_and_add(linear_ht, double_ht, add_list)  # add names from add list
            GetStatistics.search_for_100_names(linear_ht,double_ht,search_list)     # search names
            linear_count.append(linear_ht.get_probes())                             # add probe count from-
            double_count.append(double_ht.get_probes())                             # this test iteration
            if t == 999:
                print("linear hash probe counts: ", linear_count)           # print our probe count lists
                print("double hash probe counts: ", double_count)
                print("linear hash table size:         ", linear_ht.get_size())     # print table sizes
                print("double hash table size:         ", double_ht.get_size())
            linear_ht.clear()
            double_ht.clear()


        GetStatistics.compute_statistics(linear_count,double_count)  # call compute stats
        

    # i got tunnel vision and made a complicated goofy name generator
    # as a result we need an extra method to bodge things together
    @ staticmethod
    def generate_names():   # the first 676 names will be 2 char, so let's skip em for consistency
                            # then grab the 11k after those, so they're all 3 character strings
        geny = name_generator(676,11676) 
        return geny[1]      # skip the 2 letter ones, return the list of 3 letter names
    
    @staticmethod
    def choose_100_names_and_add(linear_ht, double_ht, add_list):
        # randomly chooses 100 names from the 1000
        # adds em to both linear_ht and double_ht
        one_hundred_names = random.sample(add_list, 100)    # grab a random sample of names
        for name in one_hundred_names:                      # for each name in our random sample
            linear_ht.add(name,name)                        # add to linear_ht
            double_ht.add(name,name)                        # add to double_ht
        linear_ht.reset_probe_count()                       # reset the probe count since adding might use probes
        double_ht.reset_probe_count()                       # do the same for double_ht

    @staticmethod
    def get_1000_names(names):
        return names[:1000]                                 # grab the first 1000 names from list
        
    @staticmethod
    def get_10000_names(names):
        return names[1000:]                                 # grab the next 10k names from list

    @staticmethod
    def search_for_100_names(linear_ht,double_ht,search_list):  # changed names to search_list here
        one_hundred_names = random.sample(search_list, 1000)    # grab a random sample of 100
        for name in one_hundred_names:                          # for every name in the list
            linear_ht.locate(name)                              # search linear
            double_ht.locate(name)                              # search double
    
    @staticmethod
    def compute_statistics(linear_count, dhash_count):          # this method is for 
        linear_mean = sum(linear_count) / len(linear_count)
        double_mean = sum(dhash_count) / len(dhash_count)             # get our means and print them

        linear_diffs = [(i - linear_mean) ** 2 for i in linear_count] # compute difs from mean
        double_diffs = [(i - double_mean) ** 2 for i in dhash_count]    

        l_variance = sum(linear_diffs) / (len(linear_count) - 1)      # compute variance
        d_variance = sum(double_diffs) / (len(dhash_count) -1)

        linear_standard_dev = math.sqrt(l_variance)                   # sqrt variance for std dev
        double_standard_dev = math.sqrt(d_variance)

        print("linear hash probe count mean:   ", linear_mean)        # print all our results
        # print("linear hash table size: ", GetStatistics.linear_ht.get_size())
        print("linear hash avg per search:     ", linear_mean/1000)
        print("linear hash standard deviation: ", linear_standard_dev)
        print("double hash probe count mean:   ", double_mean)
        # print("double hash table size: ", GetStatistics.double_ht.get_size())
        print("double hash avg per search:     ", double_mean/1000)
        print("double hash standard deviation: ", double_standard_dev)
        
        pass
        

if __name__ == "__main__":
    GetStatistics.main()

                            # Experiment notes below the output

linear hash probe counts:  [31566, 35781, 29045, 24954, 23538, 25002, 32463, 17918, 22450, 31992, 23756, 26395, 26827, 27027, 14331, 34993, 35018, 29021, 20255, 36115, 13309, 24780, 38419, 36034, 30535, 25165, 15381, 19044, 20985, 26557, 27549, 34096, 33402, 29919, 33368, 40035, 43223, 34824, 12633, 21840, 26837, 24756, 27269, 42491, 23775, 24764, 40127, 22578, 39748, 19973, 14221, 34948, 35993, 22098, 33274, 25484, 17995, 33345, 21679, 15673, 22048, 40351, 18569, 26935, 16393, 20613, 15232, 39008, 14381, 19200, 20523, 33653, 33210, 42437, 22725, 33831, 39550, 22118, 37204, 30612, 19924, 26420, 20242, 35324, 19350, 16687, 37463, 21347, 15702, 20780, 20107, 12751, 24106, 43175, 45946, 14656, 28585, 25496, 19031, 15971, 20346, 42023, 26292, 44437, 38479, 24880, 15903, 20390, 22153, 39893, 39788, 22211, 25166, 45732, 38671, 18147, 30811, 44890, 29893, 23456, 38661, 15384, 21997, 21689, 37306, 22405, 23945, 26694, 12493, 39899, 43176, 29766, 45094, 22389, 25779, 41450, 25843, 28815, 38037,

In [ ]:
# Below is GetStatistics test/diag code that's been cut out

# print("lcount length: ", len(linear_count))
# print("dhcount length: ", len(dhash_count))

#### TROUBLESHOOTING:
(Not all the testing was recorded here, but everything I thought important was)

    initial test results (load ratio 0.7):
        linear hash probe count mean:  50.408
        linear hash standard deviation:  10.74928291017978
        double hash probe count mean:  39.857
        double hash standard deviation:  7.694137427337177
        ####
        linear hash probe count mean:  50.322
        linear hash standard deviation:  10.033692789346851
        double hash probe count mean:  40.823
        double hash standard deviation:  7.510264647472489

    Hmmmm, I guess I'll have to tune the is_table_too_full ratio value to try and get these numbers to be more similar because it looks like double hash probing is about 20% faster here

    Test Batch 2 (load ratio 0.5):
        linear hash probe count mean:  50.374
        linear hash standard deviation:  10.614084170703697
        double hash probe count mean:  39.923
        double hash standard deviation:  7.5682461210870615
        ####
        linear hash probe count mean:  51.217
        linear hash standard deviation:  10.873092252672885
        double hash probe count mean:  40.245
        double hash standard deviation:  7.764487250441092

    Nope, double hash still seems to be 20% faster. I think that makes sense, cause the whole point of double hashing is avoiding collisions, but I guess the example results had me thinking they were going to be closer.

    I've been messing around with load ratios and table sizes for a bit trying to get the results to shift, but they've remained very close to 50 and 40 average probes respectively. I'm not sure why I'm not seeing a shift. Might add a function that computes the average number of probes per individual search.

    I think the way I currently have it set up is blowing up to the same table size everytime. I've got to change the enlarge method so that the load ratios will matter.

    That didn't seem to do anything either. Still getting the same results... That doesn't make sense.

    OKAY, finally figured out what the problem was; I need to run all the classes individually each time a change is made to any of them, I guess. Changing the table full ratio to 0.9 results in the following:

    linear hash probe count mean:  54.115
    linear hash standard deviation:  11.04190647934942
    double hash probe count mean:  46.048
    double hash standard deviation:  8.561015534794162
    
    If we want an average of at least 1.5 probes per failed search, we'd be looking at 150 probes per test, which isn't close to what I've been getting. I'm gonna start messing around with a few numbers: the initial capacity, the load threshhold for enlarging the tables, and the amount they enlarge by

    (Initial Cap: 10, Enlarge when load > 0.7, Enlarge by > 2x)
    linear hash probe count mean:  53.144
    linear hash standard deviation:  10.702981725566843
    double hash probe count mean:  45.566
    double hash standard deviation:  8.50764503396939

    (Initial Cap: 10, Enlarge when load > 0.7, Enlarge by > 1.2x)   This should have made
    linear hash probe count mean:  12.53                            the number of probes larger
    linear hash standard deviation:  3.7038923875861753             but it didn't which makes
    double hash probe count mean:  12.221                           me think something is going
    double hash standard deviation:  3.584272032924678              wrong with probe count resets

    (Initial Cap: 10, Enlarge when load > 0.7, Enlarge by > 1.2x)   No, no, wait, that doesn't make
    linear hash probe count mean:  16.564                           sense either because it only
    linear hash standard deviation:  4.519106806253458              does resets when adding; not
    double hash probe count mean:  15.733                           when searching, so that
    double hash standard deviation:  4.2555635138981085             shouldn't be a problem

    The two tests above had the same variables but different results because things aren't updating right.

    (Initial Cap: 10, Enlarge when load > 0.9, Enlarge By > 1.2x)   
    linear hash probe count mean:  16.504
    linear hash standard deviation:  4.721469715700428
    double hash probe count mean:  15.697
    double hash standard deviation:  4.523898391504462

    (Initial Cap: 10, Enlarge when load > 0.7, Enlarge By > 2x)
    linear hash probe count mean:  58.495
    linear hash standard deviation:  12.71808651978452
    double hash probe count mean:  45.12
    double hash standard deviation:  8.33781260997866

    (Initial Cap: 10, Enlarge when load > 0.95, Enlarge By > 2x)  
    linear hash probe count mean:  58.124
    linear hash standard deviation:  12.719686097322308
    double hash probe count mean:  45.002
    double hash standard deviation:  8.20983540623059

    Ok, I think something must be messed up, cause these results don't track with expected behavior. I don't think it makes sense for the number of search probes to be 3 or 4 times as high when we have a much bigger table. Either I'm having a day long brain-fart, or the enlarge and load ratio things aren't acting how I intended.

    It doesn't help that the jupiter notebook doesn't seem to update correctly unless I run all the classes individually before running GetStatistics. Even "Run All" doesn't seem to be doing it.

    The issue that has been plaguing me all day was (probably) a one character typo. I think it's fixed now

    (Initial Cap: 10, Enlarge when load > 0.7, Enlarge By > 2x)  
    linear hash probe count mean:  253.053
    linear hash standard deviation:  44.221711305090295
    double hash probe count mean:  161.879
    double hash standard deviation:  22.84894819096567

    Now double hash is close to 1.5, but linear hash is waaay off. I'm gonna start tweaking values again and see if they behave closer to expectations

    (Initial Cap: 10, Enlarge when load > 0.9, Enlarge By > 2x) 
    linear hash probe count mean:  275.845
    linear hash standard deviation:  122.125614187363
    double hash probe count mean:  166.064
    double hash standard deviation:  37.50163031491133

    (Initial Cap: 10, Enlarge when load > 0.6, Enlarge By > 2x)
    linear hash probe count mean:  60.135
    linear hash standard deviation:  23.414076834118273
    double hash probe count mean:  46.941
    double hash standard deviation:  14.173652013909873
    
    (Initial Cap: 10, Enlarge when load > 0.5, Enlarge By > 2x)
    linear hash probe count mean:  57.231
    linear hash standard deviation:  12.060061529033923
    double hash probe count mean:  45.565
    double hash standard deviation:  8.46116814431473

    There seems to be a big jump between load threshold 0.7 and 0.6 while enlarge_by > 2

    (Initial Cap: 10, Enlarge when load > 0.6, Enlarge By > 1.5x)
    linear hash probe count mean:  153.21
    linear hash standard deviation:  25.274744562758944
    double hash probe count mean:  104.477
    double hash standard deviation:  14.820226385251681

Woo, look at that. An enlarge threshold of 0.6 and enlarge factor of 1.5x+ gives Linear probing an average close to 1.5 probes per failed search, while it gives double hashing an average close to 1 probe per failed search. This makes sense since double hashing should result in less collisions than linear probing, but the problem summary says that we should expect them to be the same, which has me confused. I thought the whole point of double hashing was to avoid clumping in order to avoid collisions.

######

##### this is about where we go from troubleshooting to actually being able to run the relevant experiments

###### EXPERIMENTS:


Ok, I was trying  add some functions to collect some more data and realized the tables weren't resizing after each test so I had to fix that as well. Now results are pretty different, but we can see stuff like the end table size.

    (Initial Cap: 101, Enlarge when load > 0.7, Enlarge By > 1.5x)
    linear hash table size:          157
    double hash table size:          157
    linear hash probe count mean:    312.441
    linear hash avg per search:      3.1244099999999997
    linear hash standard deviation:  117.42401337851605
    double hash probe count mean:    172.955
    double hash avg per search:      1.7295500000000001
    double hash standard deviation:  20.977918105666443

    (Initial Cap: 101, Enlarge when load > 0.9, Enlarge By > 1.5x)
    linear hash table size:          157
    double hash table size:          157
    linear hash probe count mean:    311.38
    linear hash avg per search:      3.1138
    linear hash standard deviation:  115.14922033145923
    double hash probe count mean:    173.387
    double hash avg per search:      1.73387
    double hash standard deviation:  20.795665189906543

    (Initial Cap: 101, Enlarge when load > 0.7, Enlarge By > 2x)
    linear hash table size:          211
    double hash table size:          211
    linear hash probe count mean:    129.004
    linear hash avg per search:      1.2900399999999999
    linear hash standard deviation:  33.40922268117234
    double hash probe count mean:    89.584
    double hash avg per search:      0.8958400000000001
    double hash standard deviation:  13.175376068049614

    
    
In these last three tests we can see that while changing the threshold doesn't seem to have much effect, changing the size the table blows up by does, which makes sense because spreading things out more should lead to much less probing. I'm gonna try to fine tune for exact 1.5 values on each average, though it's seeming like double hashing is going to be able to do more with less. To get linear to 1.5 probes per search, I'm going to fine tune the enlarge_by number and I probably won't get exactly 1.5 since there are only so many primes that can be table sizes.

    (Initial Cap: 101, Enlarge when load > 0.7, Enlarge By > 1.9x)
    linear hash table size:          193
    double hash table size:          193
    linear hash probe count mean:    161.611
    linear hash avg per search:      1.61611
    linear hash standard deviation:  43.19174928393392
    double hash probe count mean:    106.491
    double hash avg per search:      1.06491
    double hash standard deviation:  14.927114179831872
    
Hmm, 1.6 probes per search is an overshoot.

    (Initial Cap: 101, Enlarge when load > 0.7, Enlarge By > 1.95x)
    linear hash table size:          199
    double hash table size:          199
    linear hash probe count mean:    145.56
    linear hash avg per search:      1.4556
    linear hash standard deviation:  36.977557244087116
    double hash probe count mean:    99.688
    double hash avg per search:      0.99688
    double hash standard deviation:  13.706513945001642

1.45 is pretty close

    (Initial Cap: 101, Enlarge when load > 0.7, Enlarge By > 1.93x)  
    linear hash table size:          197
    double hash table size:          197
    linear hash probe count mean:    150.181
    linear hash avg per search:      1.50181
    linear hash standard deviation:  36.54035353463943
    double hash probe count mean:    102.607
    double hash avg per search:      1.02607
    double hash standard deviation:  14.193801992948014


There it is. It looks like a hash table size of 197 gives us an average of 1.5 probes per failed search with the Linear Probe method. Nearly twice as many buckets as there are entries. Now I'll tune for double to get to 1.5

    (Initial Cap: 101, Enlarge when load > 0.7, Enlarge By > 1.6x) 
    linear hash table size:          163
    double hash table size:          163
    linear hash probe count mean:    268.974
    linear hash avg per search:      2.68974
    linear hash standard deviation:  85.41424305651205
    double hash probe count mean:    156.884
    double hash avg per search:      1.5688399999999998
    double hash standard deviation:  19.218113327792608

1.56 pps is already pretty close

    (Initial Cap: 101, Enlarge when load > 0.7, Enlarge By > 1.54x)
    linear hash table size:          157
    double hash table size:          157
    linear hash probe count mean:    315.024
    linear hash avg per search:      3.15024
    linear hash standard deviation:  111.91790313011158
    double hash probe count mean:    171.807
    double hash avg per search:      1.71807
    double hash standard deviation:  21.689356356959493

157 is the next prime, but it overshoots the desired 1.5 PPS. 1.56 PPS is as close as it's gonna get. With a table size of 163, we get nearly 1.5 pps with the Double Hash method. Next I'll bump up to a 1000 name search

    (Initial Cap: 101, Enlarge when load > 0.7, Enlarge By > 1.5x) Searching for 1000 names instead of 100
    linear hash table size:          157
    double hash table size:          157
    linear hash probe count mean:    3084.002
    linear hash avg per search:      3.084002
    linear hash standard deviation:  1052.9406754713496
    double hash probe count mean:    1716.576
    double hash avg per search:      1.716576
    double hash standard deviation:  68.11275630578733

Program took much longer to run when the search list was increased tenfold, but that makes sense I suppose. Runtime quadrupled to over 3 seconds rather than decupling, which suggests to me that the search is not the most time consuming process going on. I also feel like this runtime might be quite high, which makes me nervous because I've been at this for a hot minute and I haven't seen any big speed ups or errors that could be causing things to run slow. Probably the stupid name generator. The biggest difference is the standard deviation, which is like 10x for linear

    (Initial Cap: 101, Enlarge when load > 0.3, Enlarge By > 1.5x) Searching for 1000 names instead of 100
    linear hash table size:          359
    double hash table size:          359
    linear hash probe count mean:    459.243
    linear hash avg per search:      0.459243
    linear hash standard deviation:  52.311092527315864
    double hash probe count mean:    385.154
    double hash avg per search:      0.385154
    double hash standard deviation:  23.031385731838384

    (Initial Cap: 101, Enlarge when load > 0.1, Enlarge By > 1.5x) Searching for 1000 names instead of 100
    linear hash table size:          1237
    double hash table size:          1237
    linear hash probe count mean:    93.037
    linear hash avg per search:      0.09303700000000001
    linear hash standard deviation:  11.367887220581487
    double hash probe count mean:    88.213
    double hash avg per search:      0.088213
    double hash standard deviation:  9.911299857479081

The above is the only way I've found to make linear and double hashing yield similar probe counts, but it's just by making collisions super unlikely by keeping a way oversized table more than 10 times the 

    (Initial Cap: 101, Enlarge when load > 0.95, Enlarge By > 1.05x) Searching for 1000 names instead of 100
    linear hash table size:          107
    double hash table size:          107
    linear hash probe count mean:    27660.758
    linear hash avg per search:      27.660758
    linear hash standard deviation:  9022.12239941501
    double hash probe count mean:    12531.228
    double hash avg per search:      12.531227999999999
    double hash standard deviation:  412.00776985199974 

Restricting the list size to nearly the number of items doesn't help anything. This took 13.0 seconds to run


##    FINAL NOTES: 
I'm not sure if I did something wrong. The problem text says we should expect linear and double hash collision resolution to yield the same average number of probes per failed search, but that doesn't make sense to me. As stated in notes above, I thought the whole point of double hashing was to prevent the clumping that linear hashing is subject to. Since there's an algorithm to take a pseudo-random number of steps after a collision, elements in the hash table should be more spread out and we don't have to probe through a clump before finding an empty bucket. With that in mind, I think it makes sense that the double hash method didn't need nearly as many probes per failed search (hereafter reffered to as PPS). Constrainging the table size and increasing the enlarge threshold didn't make the resulting PPS averages any closer either. The data I've collected suggests that the double hashing method is able to achieve nearly 1.5 PPS with a table size of only 163, while the linear hash method needs a table of size 197 to get to 1.5 PPS. I'm unsure if I've goofed something up because of the problem text and example output that suggests they should be the same, but all of my tests have shown double hashing to use less probes, and that seems correct to me. I thought that's why we double hash.

When I increased the search list from 100 to 1000, the averages came out similar to before but interestingly standard deviation goes up quite a bit. It goes up for both, but much more so for Linear Hash which needs some explanation. I think this might be because of the exponents in the standard deviation math, which might cause a larger jump for the larger average difference, but unfortunately I haven't taken statistics yet so I could be way off there. Additionally, I think Linear Probing might have a larger deviation range because of clumping. Whether a search takes a lot of probing or not depends entirely on whether it hits a clump with its initial probe, and the larger a clump gets the more likely it is to be hit, and the more probing it's going to take to escape the clump. All that might lead to a larger range of outliers, or more distribution away from the mean. Again though, that could all be malarky because I have very little experience with statistics.

The only way I was able to make the mean probe counts similar for linear and double hash was by making the threshold for table expansion extremely low. This ensured a table with many more buckets than items which I suspect worked because it made collisions much less likely. If linear hash never makes an initial collision clumps won't form and I think that might make it perform much closer to double hash in mean probe count. Still not quite as good though, apparently.

In conclusion, Double Hashing seems like the way to go per my testing and tinkering. On average it gave much lower probe counts when searching for items within the table. I might've bungled something, but if I have I can't figure out what and I've been at it all day. That and the above reasoning for double hashing being faster makes me feel like I probably haven't goofed it up, but if I have I really must know where I went wrong.

END EXPERIMENT NOTES;

 below is the blooper reel:

In [ ]:
"""     This one has a silly flag and some other goofy mistakes
        found_key = False
        while not found_key:           # until we find the key
            if(self.table[index][0]==key):  # check if our key matches the key in that index
                found_key = True            # if it does, end loop
            elif(self.table[index] == None):# if we hit an empty
                return -1                   # -1, item not found
            else:                           # else
                # modulo get_size lets us wrap back around if we hit the end of the array
                index = (index+1) % self.get_size# increment index
        return index                        # return the index if found
"""
# When this project started, I tried for a whole different name generator that would make
# names that actually made sense rather than random strings, but it ended up being far too
# convoluted and I had to trash it. The simplified one I ended up using instead is still
# way more complicated than it needs to be becaues it brute forces through every
# possible char string of a certain length rather than just returning a list full of
# "Name_{number}" or whathaveyou

# print("EEEEEEEEEEEEEEEeeeeeeeeeeeeeeeEEEEEEEEEEEEEEE")



'\n        found_key = False\n        while not found_key:           # until we find the key\n            if(self.table[index][0]==key):  # check if our key matches the key in that index\n                found_key = True            # if it does, end loop\n            elif(self.table[index] == None):# if we hit an empty\n                return -1                   # -1, item not found\n            else:                           # else\n                # modulo get_size lets us wrap back around if we hit the end of the array\n                index = (index+1) % self.get_size# increment index\n        return index                        # return the index if found\n'